# Programming in the datbase
1. Output
    - PRINT
    - SELECT
2. Variables
3. Control-of-flow
    - IF | ELSE
    - WHILE
4. Stored Procedures
5. User Defined Functions
    - Table Valued
    - Scalar

## Output
To display some text you can use the `SELECT` and `PRINT` statement.
> Caution: There is a difference between these two statements.

In [ ]:
SELECT 'Donald Trump'; -- Notice the difference between these 2.
PRINT 'Donald Trump';

## Variables
Variables are declared in the body of a batch or procedure with the `DECLARE` statement and are assigned values by using either a `SET` or `SELECT` statement. After declaration, all variables are initialized as `NULL`, unless a value is provided as part of the declaration.
> For assigning variables, we recommend that you use `SET` `@local_variable` instead of `SELECT` `@local_variable`, since `SET` is an ANSI standard and `SELECT` for this type of operation isn't.

[Read More](https://docs.microsoft.com/en-us/sql/t-sql/language-elements/variables-transact-sql?view=sql-server-ver15)

[Why use SET instead of SELECT](https://www.c-sharpcorner.com/UploadFile/ff2f08/set-vs-select-when-assigning-values-to-variables-in-sql-serv/)

In [ ]:
-- Declaring a simple integer variable called 'myVariableName'
DECLARE @myVariableName INT
-- Declaring a simple NVARCHAR(50) variable called 'myVariableName2'
DECLARE @myVariableName2 NVARCHAR(50)

In [ ]:
DECLARE @firstName NVARCHAR(50)
SET @firstName = 'Donald'

DECLARE @lastName NVARCHAR(50) = 'Trump' -- Declaration with initialization

-- Output
PRINT @firstName + ' ' + @lastName 

In [ ]:
DECLARE
 @firstName NVARCHAR(50) = 'Donald'
,@lastName NVARCHAR(50) = 'Trump'
,@birthday DATE = '1946-06-14'

PRINT @firstName + ' ' + @lastName + ' is born at ' + CONVERT(VARCHAR,@birthday,103) -- 103 is typical for dd/MM/yyyy dateformat

In [ ]:
-- Retrieve the maximum OrderAmount of all orders and store the value in a variable.
DECLARE @max DECIMAL(7,2)
SET @max = (SELECT MAX(OrderAmount) FROM Orders) -- Notice the brackets.
PRINT @max
-- SELECT @max -- This returns a Virtual Table

# Control-of-flow
There are multiple Transact-SQL control-of-flow keywords but we'll discuss the following:
- IF...ELSE
- BEGIN...END
    - See IF...ELSE
- RETURN
    - See Stored Procedures
- THROW
    - See Deep Dive in the [workshop](https://github.com/HOGENT-Databases/DB2-Workshops/blob/master/workshops/stored-procedures/stored-procedures.md#Deep-Dive-Exception-Handling)
- TRY...CATCH
    - See Deep Dive in the [workshop](https://github.com/HOGENT-Databases/DB2-Workshops/blob/master/workshops/stored-procedures/stored-procedures.md#Deep-Dive-Exception-Handling)
- WHILE

[Read More](https://docs.microsoft.com/en-us/sql/t-sql/language-elements/if-else-transact-sql?view=sql-server-ver15)

## IF | ELSE
Imposes conditions on the execution of a Transact-SQL statement. The Transact-SQL statement that follows an `IF` keyword and its condition is executed if the condition is satisfied: the Boolean expression returns `TRUE`. The optional `ELSE` keyword introduces another Transact-SQL statement that is executed when the `IF` condition is not satisfied: the Boolean expression returns `FALSE`. 

[Read More](https://docs.microsoft.com/en-us/sql/t-sql/language-elements/if-else-transact-sql?view=sql-server-ver15)

In [ ]:
DECLARE @dayName VARCHAR(100) = DATENAME(WEEKDAY, GETDATE());
PRINT 'Current Day''s Name :' + @dayName
IF @dayName IN ('Saturday', 'Sunday')
    PRINT @dayName + ' is part of the weekend.';
    -- It's good practise to always use BEGIN and END statements.
ELSE 
    PRINT @dayName + ' is a normal working day...';
       

In [ ]:
DECLARE @birthday DATE = '1946-06-14'
IF DATEPART(YEAR,@birthday) < 2000 
    BEGIN
        PRINT 'He''s from the previous century';
    END
ELSE
    BEGIN
        PRINT 'He''s from the current century';
    END

In [ ]:
DECLARE 
 @productId INTEGER  = 1101
,@averagePrice DECIMAL(7,2) = (SELECT AVG(Price) FROM Product)  
,@productPrice DECIMAL(7,2)
,@productName VARCHAR(100)

PRINT 'The average price is ' + CONVERT(VARCHAR,@averagePrice)
SELECT 
 @productName = Product.ProductName
,@productPrice = Product.Price
FROM Product
IF @productPrice < @averagePrice
    BEGIN
        SELECT @productId       AS ProductId
            ,@productPrice    AS Price
            ,'Below average'  AS Remark
    END
ELSE  
    BEGIN
        SELECT @productId       AS ProductId
            ,@productPrice    AS Price
            ,'Above average'  AS Remark
    END

## WHILE
Sets a condition for the repeated execution of an SQL statement or statement block. The statements are executed repeatedly as long as the specified condition is true. The execution of statements in the `WHILE` loop can be controlled from inside the loop with the `BREAK` and `CONTINUE` keywords. Basically it's the same as a `WHILE` loop in any other 3rd genration language (3GL, for example Java, C#,...)
> Caution: Using `BREAK` or `CONTINUE` can make your code less readable.

[Read More](https://docs.microsoft.com/en-us/sql/t-sql/language-elements/while-transact-sql?view=sql-server-ver15)


In [ ]:
DECLARE @counter INT = 0;

WHILE @counter < 10
BEGIN
    SET @counter +=1
    PRINT @counter
END

# Stored Procedures
A stored procedure in SQL Server is a group of one or more Transact-SQL statements. Procedures resemble constructs in other programming languages because they can:
- Accept input parameters and return multiple values in the form of output parameters to the calling program.
- Contain programming statements that perform operations in the database. These include calling other procedures.
- Return a status value to a calling program to indicate success or failure (and the reason for failure).


[Read More](https://docs.microsoft.com/en-us/sql/relational-databases/stored-procedures/stored-procedures-database-engine?view=sql-server-ver15)



## Case
For our business we'd like to see which products are out of stock. So we can make sure to order some extra in the next delivery run. 

## How does the data look like?
First of all we need to know what the rows in the tables `Product` and `Supplier` contain.

In [ ]:
-- Product Table Example
SELECT TOP 3 * FROM Product
-- Supplier Table Example
SELECT TOP 3 * FROM Supplier

The query that could tackle this problem is the one below.

In [ ]:
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 250

Since this is logic we'd like to reuse we can use a stored procedure and store it as a Database Object.
> Note that a function or a view can also be used in this particular case.

## Create 
### Syntax
```sql
CREATE PROCEDURE <proc_name> [parameter declaration] 
AS
<sql_statements> 
```


In [ ]:
CREATE PROCEDURE GetProductsOutOfStock
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 250

## Execute the procedure

In [ ]:
EXECUTE GetProductsOutOfStock
-- EXEC is an alias for EXECUTE

## Delete a stored procedure


In [ ]:
DROP PROCEDURE GetProductsOutOfStock  

## Change a stored procedure
We only want to see which products are out of stock if the `UnitsInStock` is less than `100` instead of `250`

In [ ]:
ALTER PROCEDURE GetProductsOutOfStock
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 100

## Using input parameters
Since we only want to see products of a certain supplier, we'll add an `input` parameter. Since we're using parameters, a view can no longer be used.

In [ ]:
ALTER PROCEDURE GetProductsOutOfStock 
    @supplierId INT
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < 100
AND     SupplierId = @supplierId

## Execute the procedure with an input variable
> Caution: There are no brackets!


In [ ]:
EXECUTE GetProductsOutOfStock 2

## Using output parameters
Since we also want to see the name of the supplier, we'll add an `output` parameter, `supplierName`

In [ ]:
ALTER PROCEDURE GetProductsOutOfStock 
    @supplierId INT,
    @supplierName NVARCHAR(MAX) OUTPUT
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < 100
AND     SupplierId = @supplierId

SELECT @supplierName = SupplierName
FROM Supplier
WHERE SupplierId = @supplierId


## Execute the procedure with an input and output variable




In [ ]:
DECLARE @supplierName NVARCHAR(max)
EXECUTE GetProductsOutOfStock 2, @supplierName OUTPUT -- This OUTPUT statement can easily be forgotten.
PRINT 'Products out of stock for supplier:' + @supplierName

## Case
In this case we want to delete a customer based on the `CustomerNumber` but there are some checks we need to do before we delete the customer:
- The customer should exists
- The customer should't have any orders
- The INPUT parameter CustNo should be provided


In [ ]:
CREATE PROCEDURE  DeleteCustomer
 @custno INT = NULL
AS

IF @custno IS NULL
BEGIN
    PRINT 'Please provide a customerid'
    RETURN -1
END

IF NOT EXISTS (SELECT NULL FROM customer WHERE CustomerId = @custno)
BEGIN
    PRINT 'The customer doesn''t exist.'
    RETURN
END

IF EXISTS (SELECT NULL FROM Orders WHERE CustomerId = @custno)
BEGIN
    PRINT 'The customer already has orders and can''t be deleted.'
    RETURN
END

DELETE FROM Customer WHERE CustomerId = @custno
PRINT 'The customer has been succesfully deleted'

# Functions
Are basically the same as a `stored procedure` and `view` but there are some (subtle) differences:
- A function **cannot** mutate data or database objects, it can only retrieve data or make calculations based on data.
> By using a `Function` you're implicitly saying you're not mutating the database nor it's data it contains. 
- A `view` **cannot** accept input parameters, a `function` can.

## Multiple types of functions
- Scalar

    User-defined scalar functions return a single data value of the type defined in the `RETURNS` clause. For an inline scalar function, the returned scalar value is the result of a single statement. For a multistatement scalar function, the function body can contain a series of Transact-SQL statements that return the single value. The return type can be any data type except text, ntext, image, cursor, and timestamp.
- Table-Valued Functions

    User-defined table-valued functions return a table data type. For an inline table-valued function, there is no function body; the table is the result set of a single `SELECT` statement. 
- System Functions

    SQL Server provides many system functions that you can use to perform a variety of operations, do note that they cannot be modified

## Limitations
- User-defined functions cannot be used to perform actions that modify the database state.
- User-defined functions can not return multiple result sets. Use a stored procedure if you need to return multiple result sets.
- Error handling is restricted in a user-defined function. A UDF does not support TRY...CATCH, @ERROR or RAISERROR.
- User-defined functions cannot call a stored procedure.
- User-defined functions cannot make use of dynamic SQL or temp tables.
- SET statements are not allowed in a user-defined function.

[Read More](https://docs.microsoft.com/en-us/sql/relational-databases/user-defined-functions/user-defined-functions?view=sql-server-ver15)



## Case
For our business we'd like to see which products are out of stock. So we can make sure to order some extra in the next delivery run, we want to specify the minimum amount of stock and a given supplier. 

## Table Valued Function
A table-valued function is a user-defined function that returns data of a table type. The return type of a table-valued function is a table, therefore, you can use the table-valued function just like you would use a table.

In [ ]:
CREATE FUNCTION udf_GetProductsOutOfStockForSupplier(@minStock INT, @supplierId INT) RETURNS TABLE AS
RETURN 
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < @minStock
    AND SupplierId = @supplierId

## Executing a Table Valued Function
Notice that this is not possible with a SQL View.

In [ ]:
SELECT *
FROM udf_GetProductsOutOfStockForSupplier(45,7)

# Scalar Functions


In [ ]:
CREATE FUNCTION GetAge 
	(@birthdate AS DATE, @eventdate AS DATE) 
RETURNS INT 
AS 
BEGIN 
 RETURN 
 DATEDIFF(year, @birthdate, @eventdate) 
 - CASE WHEN 100 * MONTH(@eventdate) + DAY(@eventdate) < 
             100 * MONTH(@birthdate) + DAY(@birthdate) 
 THEN 1 ELSE 0 
 END; 
END;


In [ ]:
select lastname, firstname, cast(birthdate as date) birthdate,
cast(getdate() as date) today, dbo.GetAge(birthdate,getdate()) age
from employee;
